# Analyzing and visualizing
## Molecular structure
Molecular structures can be inspected with use of the `py3Dmol` module. We initialize a view and include a stick representation of the water molecule. The view is interactive, so you can rotate, translate, zoom in and out.

In [1]:
import py3Dmol as p3d

h2o_xyz = """3
water                                                                                                                          
O    0.000000000000        0.000000000000        0.000000000000                       
H    0.740848095288        0.582094932012        0.000000000000                         
H   -0.740848095288        0.582094932012        0.000000000000
"""

v = p3d.view(width=200, height=200)

v.addModel(h2o_xyz, "xyz")
v.setStyle({'stick':{}})
v.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

(section-scf-info)=
## SCF information
The SCF driver object contains a wealth of useful information:

- The final SCF energy: `scfdrv.get_scf_energy()`
- A Python dictionary of tensors: `scfdrv.scf_tensors`
    - `C`, for the molecular orbital coefficients.
    - `E`, for the orbital energies.
    - `S`, for the overlap integral.
    - `D`, for the $\alpha$- and $\beta$-spin density matrices as a tuple of NumPy arrays.
    - `F`, for the $\alpha$- and $\beta$-spin Fock matrices as a tuple of NumPy arrays.

The output from
````
print(scf_drv.scf_tensors.keys())
````

becomes
````
dict_keys(['C', 'E', 'S', 'D', 'F'])
````

So, for example, the orbital energies are available as a NumPy array by an assignment of the form
```
E = scf_drv.scf_tensors['E']
```

(section-orbitals)=
## Molecular orbitals
The values of orbitals at given coordinates are readily available. Below is an example in terms of finding the values of the $1s$-orbital ($\alpha$-spin) along the $z$-axis.  It is here assumed that the SCF optimization of the wave function has first been performed.
```
vis_drv = vlx.VisualizationDriver()
mol_orbs = scf_drv.mol_orbs

n = 100
coords = np.zeros((n,3))
coords[:,2] = np.linspace(0,3,n)

mo_1s = 0 
mo_1s_values = vis_drv.get_mo(coords, molecule, basis, mol_orbs, mo_1s, 'alpha')
```

## One- and two-particle densities
The one-particle density for an $N$-electron system is defined as

$$
    n(\mathbf{r}) = N \int 
    \Psi^\dagger(\mathbf{r}, \mathbf{r}_2, \ldots,\mathbf{r}_N)
    \Psi(\mathbf{r}, \mathbf{r}_2, \ldots,\mathbf{r}_N)
    \, d^3\mathbf{r}_2 \cdots d^3\mathbf{r}_N
$$

and it refers to the probability density of finding any one electron at position $\mathbf{r}$ in space regardless of the distribution of others. It fulfills

$$
     \int n(\mathbf{r}) \, d^3\mathbf{r} = N
$$

The two-particle density for an $N$-electron system is defined as

$$
   n(\mathbf{r}_1, \mathbf{r}_2) = N(N-1)\int 
    \Psi^\dagger(\mathbf{r}_1, \mathbf{r}_2, \ldots,\mathbf{r}_N)
    \Psi(\mathbf{r}_1, \mathbf{r}_2, \ldots,\mathbf{r}_N)
    \, d^3\mathbf{r}_3 \cdots d^3\mathbf{r}_N
$$

and it refers to the probability density of finding one electron at position $\mathbf{r}_1$ and another at position $\mathbf{r}_2$ in space regardless of the distribution of others.

For a Hartree--Fock state, the one- and two particle densities are available as illustrated in the example script below. It is here assumed that the SCF optimization of the wave function has first been performed.

```
vis_drv = vlx.VisualizationDriver()
density = scf_drv.density

# list of coordinates in units of Bohr
n = 100
coords = np.zeros((n,3))
coords[:,2] = np.linspace(0,3,n)
origin = [[0.0, 0.0, 0.0]]

one_part_den = vis_drv.get_density(coords, molecule, basis, density, 0, 'alpha')

two_part_den_aa = vis_drv.get_two_particle_density(origin, coords, molecule, basis,
                                               density, 0, 'alpha', 'alpha')
two_part_den_ab = vis_drv.get_two_particle_density(origin, coords, molecule, basis,
                                               density, 0, 'alpha', 'beta')
```

As indicated by the string argument "alpha", the $\alpha$- and $\beta$-spin densities can be determined individually, referring to the relation

$$
    n(\mathbf{r}) = n^\alpha(\mathbf{r}) + n^\beta(\mathbf{r})
$$

For a closed shell system, the two spin densities are equal and thus equal to half the total one-particle density.